# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


#### check the number of rows in csv file

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('event_data')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Query1
#### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_sessionid_iteminsession "
query = query + "(session_id int, item_in_session int, artist_name text, title text, length double, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        if(int(line[8]) == 338 and int(line[3]) ==4):
            print(int(line[8]), int(line[3]),line[0], line[9], float(line[5]))
        query = "INSERT into songs_sessionid_iteminsession (session_id, item_in_session, artist_name, title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

338 4 Faithless Music Matters (Mark Knight Dub) 495.3073


#### Do a SELECT to verify the result for Query 1

In [10]:
query = """
    SELECT artist_name, title, length From songs WHERE session_id=338 AND item_in_session=4;
"""

try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)
    


Error from server: code=2200 [Invalid query] message="unconfigured table songs"


### Query 2: 
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182              

In [11]:
query = """
CREATE TABLE  songs_sessionid_userid(
    session_id int, 
    item_in_session int, 
    user_id int, 
    first_name text, 
    last_name text, 
    artist_name text, 
    title text, 
    length float, 
  PRIMARY KEY ((session_id, user_id), item_in_session, first_name, last_name)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        if(int(line[8]) == 182 and int(10) == 10):
                print(int(line[8]), int(line[3]), int(line[10]), line[1], line[4],  line[0], line[9], float(line[5]))
        query = "INSERT into songs_sessionid_userid (session_id, item_in_session, user_id, first_name, last_name, artist_name, title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), int(line[10]), line[1], line[4],  line[0], line[9], float(line[5]) ))

182 0 10 Sylvie Cruz Down To The Bone Keep On Keepin' On 333.76608
182 1 10 Sylvie Cruz Three Drives Greece 2000 411.6371
182 2 10 Sylvie Cruz Sebastien Tellier Kilometer 377.73016
182 3 10 Sylvie Cruz Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 181.21098


#### Do a SELECT to verify the result for Query 2

In [13]:
query = """
  SELECT item_in_session, first_name, last_name, artist_name, title 
  FROM songs_sessionid_userid 
  WHERE session_id=182 AND user_id=10
  ORDER BY item_in_session, first_name, last_name;
"""

try:
    rows = session.execute(query)
    print(rows)
    for row in rows:
        print(row)
except Exception as e:
    print (e)
    


Row(item_in_session=0, first_name='Sylvie', last_name='Cruz', artist_name='Down To The Bone', title="Keep On Keepin' On")
Row(item_in_session=1, first_name='Sylvie', last_name='Cruz', artist_name='Three Drives', title='Greece 2000')
Row(item_in_session=2, first_name='Sylvie', last_name='Cruz', artist_name='Sebastien Tellier', title='Kilometer')
Row(item_in_session=3, first_name='Sylvie', last_name='Cruz', artist_name='Lonnie Gordon', title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


### Query 3: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                    

In [14]:
query = """
CREATE TABLE songs_title (
    title text,
    user_id int,
    first_name text,
    last_name text,
  PRIMARY KEY(title, user_id)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
query = """
    INSERT INTO songs_title 
    (title, user_id, first_name, last_name)
    VALUES (%s, %s, %s, %s)
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[9], int(line[10]), line[1], line[4] ))
        if(line[9] == 'All Hands Against His Own'):
            print(line)

['The Black Keys', 'Tegan', 'F', '25', 'Levine', '196.91057', 'paid', 'Portland-South Portland, ME', '611', 'All Hands Against His Own', '80']
['The Black Keys', 'Sara', 'F', '31', 'Johnson', '196.91057', 'paid', 'Winston-Salem, NC', '152', 'All Hands Against His Own', '95']
['The Black Keys', 'Jacqueline', 'F', '50', 'Lynch', '196.91057', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '559', 'All Hands Against His Own', '29']


#### Do a SELECT to verify the result for Query 3

In [16]:
query = """
  SELECT first_name, last_name 
  FROM songs_title
  WHERE title = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query);
    for row in rows:
        print(row)
except Exception as e:
    print(e)
    

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [17]:
query1 = "DROP TABLE songs_sessionid_iteminsession;"
query2 = "DROP TABLE songs_sessionid_userid"
query3 = "DROP TABLE songs_title"
try:
    session.execute(query1);
    session.execute(query2);
    session.execute(query3);
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()